# Raw Task Injection

Wrap external `nidaqmx.task.Task` objects for interop with existing code.

In [ ]:
# Check for nidaqmx
try:
    import nidaqmx
except ImportError:
    raise RuntimeError("nidaqmx not installed. Install with: pip install nidaqmx")

In [ ]:
import numpy as np
from nidaqwrapper import AITask, AOTask, DITask, DOTask, DAQHandler
from nidaqwrapper.utils import list_devices

# Show connected hardware
devices = list_devices()
print("Connected devices:")
for idx, dev in enumerate(devices):
    print(f"  {idx}: {dev['name']} ({dev['product_type']})")

print("\nNote: This notebook requires real hardware to run.")

## Wrap External AI Task

In [ ]:
# Create raw nidaqmx task with low-level API
raw_task = nidaqmx.Task("external_ai")
raw_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
raw_task.timing.cfg_samp_clk_timing(rate=25600)

# Wrap with AITask
wrapped = AITask.from_task(raw_task)
print(f"Wrapped task: {wrapped.task_name}")
print(f"Channels: {wrapped.channel_list}")
print(f"Sample rate: {wrapped.sample_rate} Hz")

# Start and acquire with blocking read
raw_task.start()
data = wrapped.acquire(n_samples=1000)  # Blocks until 1000 samples ready
print(f"Acquired: {data.shape}")

# Caller closes raw task (wrapper does NOT close external tasks)
raw_task.close()

## Wrap External DO Task

In [ ]:
# Create raw digital output task
raw_do = nidaqmx.Task("external_do")
raw_do.do_channels.add_do_chan("Dev1/port0/line0:3")

# Wrap with DOTask
wrapped_do = DOTask.from_task(raw_do)
print(f"Wrapped DO task: {wrapped_do.task_name}")
print(f"Channels: {wrapped_do.channel_list}")

# Write data
wrapped_do.write([True, False, True, False])

# Caller closes
raw_do.close()

## Raw Task in DAQHandler

In [ ]:
# Pass raw nidaqmx.Task to DAQHandler.configure()
raw_ai = nidaqmx.Task("handler_raw")
raw_ai.ai_channels.add_ai_voltage_chan("Dev1/ai0")
raw_ai.timing.cfg_samp_clk_timing(rate=25600)

handler = DAQHandler()
handler.configure(task_in=raw_ai)
handler.connect()

# Use handler's read method with blocking read
raw_ai.start()
data = handler.read_all_available(n_samples=1000)  # Blocks until 1000 samples ready
print(f"Read from handler: {data.shape}")

handler.disconnect()
raw_ai.close()

## Ownership

When created via `from_task()`:
- `_owns_task = False` flag is set
- `clear_task()` does NOT close the underlying task
- Caller retains full ownership and must call `task.close()`

This allows safe interop with external nidaqmx code.

## When to Use

Raw task injection is useful for:
- Integrating with existing nidaqmx code
- Accessing low-level features not exposed by wrapper
- Migrating legacy code incrementally

For new code, use the wrapper's native API (add_channel, start, etc).